# Unsere erste Auswertung
## Inhalt der Tabelle Station
Wir lesen direkt von der SQL Server Datenbank. Das Kennwort ist hier gespeichert, das sollte natürlich
nicht veröffentlicht werden.

In [3]:
import pyodbc
import sqlalchemy

connection_url = sqlalchemy.engine.URL.create(
    "mssql+pyodbc",
    username="sa",
    password="1234",
    host=".\SQLSERVER2019",
    database="Fahrkarten",
    query={
        "driver": "ODBC Driver 17 for SQL Server"
    },
)

engine = sqlalchemy.create_engine(connection_url)
with engine.connect() as conn:
    result = conn.execute("SELECT * FROM Station")
    records = result.fetchall()
    for row in records:
        print(row[0], row[1], row[2])


1000 Meidling 48.175000000000000
1001 Westbahnhof 48.196666666666667
1002 Längenfeldgasse 48.185000000000000
1003 Praterstern 48.218333333333333


## Der Dataframe
Wir können die Daten auch in eine spezielle Struktur lesen: dem Dataframe.

In [4]:
import pandas as pd
with engine.connect() as conn:
    stations = pd.read_sql("SELECT * FROM Station", conn)
print(stations)

   StationId             Name   Latitude  Longitude
0       1000         Meidling  48.175000  16.335000
1       1001      Westbahnhof  48.196667  16.338333
2       1002  Längenfeldgasse  48.185000  16.335000
3       1003      Praterstern  48.218333  16.391667


Der Dataframe ist eine Collection, die einen flexiblen Zugriff auf die Daten ermöglicht. So können
wir nur eine Spalte ausgeben. Da das folgende Statement das letzte Statement ist, wird es automatisch
ausgegeben:

In [5]:
stations["Name"]  # oder stations.Name

0           Meidling
1        Westbahnhof
2    Längenfeldgasse
3        Praterstern
Name: Name, dtype: object

### Filtern im Dataframe
Die einfachste Filterung ist eine Auswahl der Datensätze. So werden die ersten 2 Datensätze
ausgegeben. Achtung: Es wird bei 0 begonnen zu wählen. Der 2. Index (hier ist er 2) ist *exklusice*.
Das ist die sog. *slice syntax*.

In [6]:
stations[0:2]

,StationId,Name,Latitude,Longitude
0,1000,Meidling,48.175000,16.335000
1,1001,Westbahnhof,48.196667,16.338333


Wir können auch einen Ausdruck in die eckicken Klammern schreiben.

In [7]:
stations[stations["Latitude"] > 48.18]

,StationId,Name,Latitude,Longitude
1,1001,Westbahnhof,48.196667,16.338333
2,1002,Längenfeldgasse,48.185000,16.335000
3,1003,Praterstern,48.218333,16.391667


Sehen wir uns den folgenden Ausdruck genauer an. Er liefert eine Liste mit
*False, True, True, True*. Der > Operator ist also "überladen", denn stations["Latitude"] ist eine
Liste an Werten. Diese Liste wird verglichen und liefert eine Liste zurück.

In [8]:

stations["Latitude"] > 48.18

0    False
1     True
2     True
3     True
Name: Latitude, dtype: bool

Mehrere Filter können wir mit & kombinieren. Wichtig ist die Klammerung.

In [9]:
stations[(stations["Latitude"] > 48.18) & (stations["Longitude"] > 16.35)]

,StationId,Name,Latitude,Longitude
3,1003,Praterstern,48.218333,16.391667


Das folgende Beispiel funktioniert jedoch nicht. startswith ist eine Methode, die für Strings
definiert ist. Stations.Name (wir können auch mit Punkt auf eine Spalte zugreifen) ist eine Liste
(genau genommen der Typ Series).

In [10]:
# stations[stations.Name.startswith("N")]

Deswegen gibt es in Pandas für Reihen (Series) das Property str. Es liefert eine Stringliste
zurück, die Filterfunktionen anbietet. Siehe https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html

In [11]:
stations[stations.Name.str.startswith("M")]

,StationId,Name,Latitude,Longitude
0,1000,Meidling,48.175,16.335


Möchten wir auch die zurückgegebenen Spalten festlegen, können wir die Funktion *loc* verwenden.
Als zweiten Parameter geben wir eine Liste von Spalten, die wir ausgeben möchten, an. In Python gibt es
keine "klassischen" Arrays. Mit der eckigen Klammer können wir eine Liste definieren.

In [12]:
stations.loc[stations["Latitude"] > 48.18, ["Name", "Latitude"]]

,Name,Latitude
1,Westbahnhof,48.196667
2,Längenfeldgasse,48.185000
3,Praterstern,48.218333


Da ein Subset eines Dataframes wieder ein Dataframe ist, können wir einen Filter mit den
Methoden max(), min(), ... kombinieren. Sie liefern das Maximum, Minimum, ... jeder Spalte.
Da wir jede Zeile ausgeben möchten, brauchen wir print.

In [13]:
print(stations.loc[stations["Latitude"] > 48.18, ["Latitude", "Longitude"]].min())
print(stations.loc[stations["Latitude"] > 48.18, ["Latitude", "Longitude"]].mean())
print(stations.loc[stations["Latitude"] > 48.18, ["Latitude", "Longitude"]].max())

Latitude     48.185
Longitude    16.335
dtype: float64
Latitude     48.200
Longitude    16.355
dtype: float64
Latitude     48.218333
Longitude    16.391667
dtype: float64


Wir können mit Sort in Verbindung mit der slice Syntax sehr leicht die 3 südlichsten Haltestellen
herausfinden:

In [14]:
stations.sort_values("Latitude")[0:3]

,StationId,Name,Latitude,Longitude
0,1000,Meidling,48.175000,16.335000
2,1002,Längenfeldgasse,48.185000,16.335000
1,1001,Westbahnhof,48.196667,16.338333


Wir können auch neue Spalten anlegen und sie mit einem Wert befüllen. Beachte, dass rechts von der
Zuweisung eine ganze Liste erzeugt wird.

In [15]:
stations["NewName"] = stations.Name + "A"
print(stations)

   StationId             Name   Latitude  Longitude           NewName
0       1000         Meidling  48.175000  16.335000         MeidlingA
1       1001      Westbahnhof  48.196667  16.338333      WestbahnhofA
2       1002  Längenfeldgasse  48.185000  16.335000  LängenfeldgasseA
3       1003      Praterstern  48.218333  16.391667      PratersternA
